In [14]:
%env LS_COLORS = None 
!tree --charset ascii  /home/yu/Templates/AE-VAE-GAN/DataLoder/data/dogcat/

env: LS_COLORS=None
/home/yu/Templates/AE-VAE-GAN/DataLoder/data/dogcat/
|-- cat.12484.jpg
|-- cat.12485.jpg
|-- cat.12486.jpg
|-- cat.12487.jpg
|-- dog.12496.jpg
|-- dog.12497.jpg
|-- dog.12498.jpg
`-- dog.12499.jpg

0 directories, 8 files


__getitem__:返回一条数据或者一个样本
__len__:返回样本的数量

In [20]:
import os
from PIL import Image
import numpy as np
import torch
from torch.utils import data

class DogCat(data.Dataset):
    def __init__(self, root):
        imgs = os.listdir(root)
        self.imgs = [os.path.join(root, img) for img in imgs]
    
    def __getitem__(self, index):
        img_path = self.imgs[index]
        label = 1 if 'dog' in img_path.split('/')[-1] else 0
        pil_img = Image.open(img_path)
        array = np.asarray(pil_img)
        data = torch.from_numpy(array)
        return data, label
    
    def __len__(self):
        return len(self.imgs)

In [21]:
dataset = DogCat('./data/dogcat/')
img, label = dataset[0] #相当于调用__getitem__(0)
for img, label in dataset:
    print(img.size(), img.float().mean(), label)

torch.Size([236, 289, 3]) 130.30038805153168 0
torch.Size([375, 499, 3]) 150.50795635715878 1
torch.Size([375, 499, 3]) 116.81384992206635 1
torch.Size([400, 300, 3]) 128.154975 1
torch.Size([499, 379, 3]) 171.80847887507645 0
torch.Size([377, 499, 3]) 151.7174171508357 1
torch.Size([500, 497, 3]) 106.4915063715627 0
torch.Size([374, 499, 3]) 115.51768778198108 0
